In [39]:
import cv2
import numpy as np

MHI_DURATION = 5 # Duration of motion history
THRESHOLD = 30   # Threshold for motion detection

cap1 = cv2.VideoCapture('person1_gesture7_com.avi')
cap2 = cv2.VideoCapture('person2_gesture5_com.avi')
cap3 = cv2.VideoCapture('person4_gesture7_com.avi')

videos = ['person1_gesture7_com.avi', 'person2_gesture5_com.avi', 'person4_gesture7_com.avi']

fps1 = cap1.get(cv2.CAP_PROP_FPS)
fps2 = cap2.get(cv2.CAP_PROP_FPS)
fps3 = cap3.get(cv2.CAP_PROP_FPS)

num_frames1 = int(fps1 * 3)
num_frames2 = int(fps2 * 2)
num_frames3 = int(fps3 * 4)

total_frames = [num_frames1,num_frames2,num_frames3]

index = 0
# Loop through the videos
for video in videos:
    # Creating a video capture for the input video
    cap = cv2.VideoCapture(video)
    num_frames = total_frames[index]
    index += 1
    
    # Getting the first frame of the video
    ret, frame = cap.read()

    if not ret:
        print(f"Error: Unable to read the first frame of {video}")
    else:
        # Converting the frame to grayscale and initializing the MHI
        prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mhi = np.zeros_like(prev_frame)

        # Loop through each frames of the video
        for i in range(num_frames):
            # Read the next frame
            ret, frame = cap.read()

            # If we've reached the end of the video, break out of the loop
            if not ret:
                break

            # Converting the frame to grayscale
            curr_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Computing the difference between the current frame and the previous frame
            diff = cv2.absdiff(curr_frame, prev_frame)

            # cv2.threshold, pixels with intensities above the threshold are set to 1 and those with intensities below the threshold are set to 0
            _, thresh = cv2.threshold(diff, THRESHOLD, 1, cv2.THRESH_BINARY)

            # Updating the motion history image
            mhi[thresh != 0] = MHI_DURATION

            # Decreasing the MHI values over time
            mhi[mhi > 0] -= 1

            # Copying the current frame to the previous frame
            prev_frame = curr_frame.copy()

            # Display the current frame and the MHI 
            cv2.imshow('frame', frame)
            cv2.imshow('mhi', np.uint8(np.clip((mhi / float(MHI_DURATION)) * 255, 0, 255)))
            
            #Saving the motion history image
            if i == num_frames - 1:
                filename = f'{video}_mhi.jpg'
                cv2.imwrite(filename, np.uint8(np.clip((mhi / float(MHI_DURATION)) * 255, 0, 255)))

            # Waiting for a key press
            if cv2.waitKey(30) == ord('q'):
                break

    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()
